In [19]:
import sys
import os
from datetime import datetime, timedelta, timezone
import pandas as pd

# Add src directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src import config
from src.data_utils import load_and_process_citibike_data, transform_raw_data_into_ts_data


In [20]:
from src.data_utils import load_and_process_citibike_data, transform_raw_data_into_ts_data, load_and_process_citibike_data_local
from hsfs.feature import Feature
from datetime import timedelta

In [21]:
from datetime import datetime, timedelta
import pandas as pd
from src.data_utils import load_and_process_citibike_data_local
import src.config as config

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Step 1: Load last 13 months of local Citi Bike data
    all_rides = load_and_process_citibike_data_local(
        base_path=config.LOCAL_CITIBIKE_DATA_PATH,
        months_back=13
    )

    # Step 2: Filter by time window
    all_rides['started_at'] = pd.to_datetime(all_rides['started_at']).dt.tz_localize(None)
    filtered_rides = all_rides[
        (all_rides['started_at'] >= from_date.replace(tzinfo=None)) &
        (all_rides['started_at'] < to_date.replace(tzinfo=None))
    ]

    # Step 3: Sort
    filtered_rides.sort_values(by=['start_station_id', 'started_at'], inplace=True)

    return filtered_rides

In [22]:
# def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
#     if from_date >= to_date:
#         raise ValueError("'from_date' must be earlier than 'to_date'.")

#     historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
#     historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

#     rides_from = load_and_process_citibike_data_from_local(
#         year=historical_from_date.year, months=[historical_from_date.month]
#     )
#     rides_from['started_at'] = rides_from['started_at'].dt.tz_localize(None)
#     rides_from = rides_from[rides_from.started_at >= historical_from_date]

#     if historical_to_date.month != historical_from_date.month:
#         rides_to = load_and_process_citibike_data_from_local(
#             year=historical_to_date.year, months=[historical_to_date.month]
#         )
#         rides_to['started_at'] = rides_to['started_at'].dt.tz_localize(None)
#         rides_to = rides_to[rides_to.started_at < historical_to_date]
#         rides = pd.concat([rides_from, rides_to], ignore_index=True)
#     else:
#         rides = rides_from

#     rides['started_at'] += timedelta(weeks=52)
#     rides.sort_values(by=['start_station_id', 'started_at'], inplace=True)

#     return rides


In [23]:
from datetime import timedelta
import pandas as pd

# Last full day in the dataset is April 30, 2025
fetch_data_to = pd.Timestamp("2025-04-30 00:00:00", tz="UTC")

# Fetch the 30-day window ending at that point
fetch_data_from = fetch_data_to - timedelta(days=29)

In [24]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)
print("✅ Raw data shape:", rides.shape)
rides.head()

⚠️ No file found for 202505
📂 Reading: JC-202504-citibike-tripdata.csv
📂 Reading: JC-202503-citibike-tripdata.csv
📂 Reading: JC-202502-citibike-tripdata.csv
📂 Reading: JC-202501-citibike-tripdata.csv
📂 Reading: JC-202412-citibike-tripdata.csv
📂 Reading: JC-202411-citibike-tripdata.csv
📂 Reading: JC-202410-citibike-tripdata.csv
📂 Reading: JC-202409-citibike-tripdata.csv
📂 Reading: JC-202408-citibike-tripdata.csv
📂 Reading: JC-202407-citibike-tripdata.csv
📂 Reading: JC-202406-citibike-tripdata.csv
📂 Reading: JC-202405-citibike-tripdata.csv
✅ Loaded and processed 12 months. Total records: 1,051,856


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


✅ Raw data shape: (78057, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
76245,71B146CEBBD95B98,electric_bike,2025-04-01 06:27:43.810,2025-04-01 06:35:26.418,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Christ Hospital,JC034,40.735938,-74.030305,40.734786,-74.050444,member
79242,58852E8BB390272D,classic_bike,2025-04-01 08:33:01.454,2025-04-01 08:38:40.304,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Madison St & 1 St,HB402,40.735938,-74.030305,40.738790,-74.039300,member
78957,D62A14D5B64E3A3B,electric_bike,2025-04-01 09:00:53.294,2025-04-01 09:05:50.312,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Clinton St & 7 St,HB303,40.735938,-74.030305,40.745420,-74.033320,casual
78126,D91A64B6C9C3D1AC,electric_bike,2025-04-01 09:28:06.305,2025-04-01 09:30:58.412,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
78149,C228188CA68B62B1,electric_bike,2025-04-01 09:44:45.804,2025-04-01 09:49:22.162,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member


In [25]:
rides = rides.rename(columns={"started_at": "pickup_datetime", "start_station_id": "pickup_location_id"})

In [26]:
print(rides.columns)
rides.head()

Index(['ride_id', 'rideable_type', 'pickup_datetime', 'ended_at',
       'start_station_name', 'pickup_location_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


,ride_id,rideable_type,pickup_datetime,ended_at,start_station_name,pickup_location_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
76245,71B146CEBBD95B98,electric_bike,2025-04-01 06:27:43.810,2025-04-01 06:35:26.418,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Christ Hospital,JC034,40.735938,-74.030305,40.734786,-74.050444,member
79242,58852E8BB390272D,classic_bike,2025-04-01 08:33:01.454,2025-04-01 08:38:40.304,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Madison St & 1 St,HB402,40.735938,-74.030305,40.738790,-74.039300,member
78957,D62A14D5B64E3A3B,electric_bike,2025-04-01 09:00:53.294,2025-04-01 09:05:50.312,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Clinton St & 7 St,HB303,40.735938,-74.030305,40.745420,-74.033320,casual
78126,D91A64B6C9C3D1AC,electric_bike,2025-04-01 09:28:06.305,2025-04-01 09:30:58.412,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
78149,C228188CA68B62B1,electric_bike,2025-04-01 09:44:45.804,2025-04-01 09:49:22.162,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member


In [27]:
rides

,ride_id,rideable_type,pickup_datetime,ended_at,start_station_name,pickup_location_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
76245,71B146CEBBD95B98,electric_bike,2025-04-01 06:27:43.810,2025-04-01 06:35:26.418,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Christ Hospital,JC034,40.735938,-74.030305,40.734786,-74.050444,member
79242,58852E8BB390272D,classic_bike,2025-04-01 08:33:01.454,2025-04-01 08:38:40.304,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Madison St & 1 St,HB402,40.735938,-74.030305,40.738790,-74.039300,member
78957,D62A14D5B64E3A3B,electric_bike,2025-04-01 09:00:53.294,2025-04-01 09:05:50.312,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Clinton St & 7 St,HB303,40.735938,-74.030305,40.745420,-74.033320,casual
78126,D91A64B6C9C3D1AC,electric_bike,2025-04-01 09:28:06.305,2025-04-01 09:30:58.412,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
78149,C228188CA68B62B1,electric_bike,2025-04-01 09:44:45.804,2025-04-01 09:49:22.162,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34748,4050DCA801E1BE39,classic_bike,2025-04-29 19:32:06.852,2025-04-29 19:50:21.571,Exchange Pl,JC116,4 St & River St,HB611,40.716366,-74.034344,40.740814,-74.027406,member
14754,DACB9FE2CDEEF36C,electric_bike,2025-04-29 19:52:02.388,2025-04-29 19:55:10.529,Exchange Pl,JC116,Marin Light Rail,JC013,40.716366,-74.034344,40.714584,-74.042817,member
58930,EDDED2754E9E41CD,electric_bike,2025-04-29 20:39:56.598,2025-04-29 20:55:20.124,Exchange Pl,JC116,JC Medical Center,JC110,40.716366,-74.034344,40.715391,-74.049692,casual
28113,4994150403108317,classic_bike,2025-04-29 20:52:44.870,2025-04-29 20:55:14.164,Exchange Pl,JC116,Essex Light Rail,JC038,40.716366,-74.034344,40.712774,-74.036486,member


In [28]:
import pandas as pd

# Ensure proper datetime format
rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'], errors='coerce')

# Get top 5 most common pickup locations
top_station_ids = (
    rides["pickup_location_id"]
    .value_counts()
    .head(5)
    .index
    .tolist()
)

print("Top 5 station IDs:", top_station_ids)

# Filter only those top station rides
top_rides = rides[rides['pickup_location_id'].isin(top_station_ids)].copy()

# Add year column
top_rides['year'] = top_rides['pickup_datetime'].dt.year

# Group by station and year
ride_counts = (
    top_rides
    .groupby(['pickup_location_id', 'year'])
    .size()
    .reset_index(name='ride_count')
    .sort_values(['pickup_location_id', 'year'])
)

# Output
print("📊 Ride counts for top 5 start stations by year:")
print(ride_counts.to_string(index=False))


Top 5 station IDs: ['HB102', 'JC115', 'HB105', 'HB101', 'JC009']
📊 Ride counts for top 5 start stations by year:
pickup_location_id  year  ride_count
             HB101  2025        1848
             HB102  2025        3955
             HB105  2025        2002
             JC009  2025        1776
             JC115  2025        3443


In [29]:
ts_data = transform_raw_data_into_ts_data(rides)
print("✅ Transformed time-series shape:", ts_data.shape)
ts_data.head()


✅ Transformed time-series shape: (57768, 3)


,pickup_hour,pickup_location_id,rides
0,2025-04-01 00:00:00,HB101,0
1,2025-04-01 01:00:00,HB101,0
2,2025-04-01 02:00:00,HB101,0
3,2025-04-01 03:00:00,HB101,0
4,2025-04-01 04:00:00,HB101,0


In [30]:
# Keep only top 5 station IDs
ts_data = ts_data[ts_data['pickup_location_id'].isin(top_station_ids)].copy()

# Convert pickup_location_id and rides to correct dtypes (int)
# ts_data['pickup_location_id'] = pd.to_numeric(ts_data['pickup_location_id'], errors='coerce').astype(float).astype(int)
ts_data['rides'] = pd.to_numeric(ts_data['rides'], errors='coerce').astype(int)


In [31]:
# ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype("int32")
ts_data['rides'] = ts_data['rides'].astype("int64")


In [32]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

fs = project.get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

fg.insert(ts_data, write_options={"wait_for_job": False})


2025-05-10 10:08:56,848 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:08:56,895 INFO: Initializing external client
2025-05-10 10:08:56,898 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 10:08:58,018 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


Uploading Dataframe: 100.00% |██████████| Rows 3480/3480 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215665/jobs/named/citibike_hourly_features_1_offline_fg_materialization/executions


(Job('citibike_hourly_features_1_offline_fg_materialization', 'SPARK'), None)

###  Completed

In [33]:
import hopsworks
from hsfs.feature import Feature

project = hopsworks.login()
fs = project.get_feature_store()

# Drop existing feature group if it exists
try:
    fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)
    fg.delete()
    print("✅ Deleted old feature group.")
except:
    print("ℹ️ No existing feature group to delete.")

# Define new schema
features = [
    Feature(name="pickup_hour", type="timestamp"),
    Feature(name="pickup_location_id", type="string"),  # Changed from int
    Feature(name="rides", type="int"),
]

# Create new feature group
fg = fs.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Hourly aggregated Citi Bike rides per location",
    primary_key=["pickup_hour", "pickup_location_id"],
    event_time="pickup_hour",
    features=features,
    online_enabled=True
)

print("✅ Created new feature group with correct schema.")


2025-05-10 10:09:21,667 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:09:21,694 INFO: Initializing external client
2025-05-10 10:09:21,696 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:09:23,045 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
ℹ️ No existing feature group to delete.
✅ Created new feature group with correct schema.


In [34]:
import hopsworks
import src.config as c           # has the FEATURE_VIEW_NAME / VERSION you use

project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

print("✅  Connected. Feature‑store id:", fs.id)

2025-05-10 10:09:31,767 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:09:31,781 INFO: Initializing external client
2025-05-10 10:09:31,782 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:09:32,998 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
✅  Connected. Feature‑store id: 1203294


In [35]:
from hsfs.feature import Feature

NAME    = c.FEATURE_VIEW_NAME        # "time_series_hourly_feature_view"
OLD_VER = c.FEATURE_VIEW_VERSION     # 1
NEW_VER = OLD_VER + 1                # 2

# ── A) Delete the stale v1 (ignore error if it’s already gone)
try:
    fs.get_feature_view(NAME, version=OLD_VER).delete()
    print(f"🗑️  Deleted old feature‑view {NAME} v{OLD_VER}")
except Exception as e:
    print(f"ℹ️  Could not delete v{OLD_VER} (probably already gone).")

# ── B) Point the view at the new feature‑group you created
fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = NAME,
    version     = NEW_VER,
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print(f"✅  Feature‑view rebuilt: {fv.name}  v{fv.version}")


ℹ️  Could not delete v1 (probably already gone).


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [36]:
# --- reconnect (fresh session is safest) -----------------
project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# --- choose the next clean version -----------------------
NAME     = c.FEATURE_VIEW_NAME          # "time_series_hourly_feature_view"
NEW_VER  = 3                            # skip v2 that failed

fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = NAME,
    version     = NEW_VER,
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print(f"✅  Feature‑view created: {fv.name}  v{fv.version}")


2025-05-10 10:10:04,904 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:10:04,924 INFO: Initializing external client
2025-05-10 10:10:04,928 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:10:06,072 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [37]:
import hopsworks, src.config as c

project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

NEW_NAME = "cb_hourly_feature_view"     # any fresh name
NEW_VER  = 1                            # start at 1 again

fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = NEW_NAME,
    version     = NEW_VER,
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print(f"✅  Feature‑view created: {fv.name}  v{fv.version}")


2025-05-10 10:10:08,958 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:10:08,967 INFO: Initializing external client
2025-05-10 10:10:08,968 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:10:10,209 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [ ]:
# list every feature‑view in the store
views = fs.get_feature_views(name=None)   # ← pass name=None
for v in views:
    # .id may not be exposed in some versions; print status too
    print(f"{v.name:<35}  v{v.version:<2}  status={getattr(v, 'status', '‑')}  id={getattr(v, 'id', '‑')}")


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1214683/featurestores/1202314/featureview/None). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":120004,"usrMsg":"HTTP 404 Not Found","errorMsg":"Web application exception occurred"}', error code: 120004, error msg: Web application exception occurred, user msg: HTTP 404 Not Found

In [ ]:
VIEW_NAME = c.FEATURE_VIEW_NAME          # "time_series_hourly_feature_view"

views = fs.get_feature_views(name=VIEW_NAME)   # returns *all* versions for that name
for v in views:
    print(f"{v.name:<35}  v{v.version:<3}   status={getattr(v,'status','‑')}")


ValueError: Cannot get back the feature view because the query defined is no longer valid. Some feature groups used in the query may have been deleted. You can clean up this feature view on the UI or `FeatureView.clean`.

In [39]:
from hsfs.client.exceptions import RestAPIError

VIEW = c.FEATURE_VIEW_NAME          # "time_series_hourly_feature_view"

# brute‑force try versions 1‑10 and hard‑delete if they exist
for v in range(1, 11):
    try:
        fv = fs.get_feature_view(VIEW, version=v)
        print(f"🗑️  found {VIEW} v{v} – cleaning …")
        fv.clean(hard_delete=True)          # <- *really* removes metadata
        print(f"   → deleted v{v}")
    except (ValueError, RestAPIError):
        pass                                # view/version not present / already invalid


In [40]:
from hsfs.client.exceptions import RestAPIError

VIEW = c.FEATURE_VIEW_NAME          # "time_series_hourly_feature_view"

for v in range(1, 11):
    try:
        fv = fs.get_feature_view(VIEW, version=v)
        if fv is None:                       # ← skip missing / corrupt entries
            continue
        print(f"🗑️  found {VIEW} v{v} – cleaning …")
        fv.clean(hard_delete=True)
        print(f"   → deleted v{v}")
    except (ValueError, RestAPIError):
        pass


In [41]:
# still in the same notebook / Python session
fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = c.FEATURE_VIEW_NAME,     # "time_series_hourly_feature_view"
    version     = 1,                       # brand‑new v1
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print("✅  Feature‑view ready:", fv.name, "v", fv.version)


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [42]:
from hsfs.core.feature_view_api import FeatureViewApi
from hopsworks.client.exceptions import RestAPIError

VIEW_NAME = c.FEATURE_VIEW_NAME    # "time_series_hourly_feature_view"
VIEW_VER  = 1

# get the numeric id of the current feature‑store
store_id = fs.id            # <‑─ this exists on the FeatureStore object
# (in older hsfs versions it’s `fs._id`; use whichever is present)

_fv_api = FeatureViewApi(store_id)

try:
    print(f"🔍 trying hard‑delete of {VIEW_NAME} v{VIEW_VER} …")
    _fv_api.delete(name=VIEW_NAME, version=VIEW_VER, hard_delete=True)
    print("✅ hard‑deleted")
except RestAPIError:
    print("ℹ️  nothing to delete – continuing")


🔍 trying hard‑delete of time_series_hourly_feature_view v1 …


AttributeError: 'FeatureViewApi' object has no attribute 'delete'

In [43]:
from hsfs.client.exceptions import RestAPIError

VIEW = c.FEATURE_VIEW_NAME          # "time_series_hourly_feature_view"

for v in range(1, 11):               # try v1 … v10
    try:
        fv = fs.get_feature_view(VIEW, version=v)
        print(f"🗑️  cleaning {VIEW} v{v} …")
        fv.clean(hard_delete=True)   # <-- really drops metadata
    except (ValueError, RestAPIError):
        # either the view/version doesn’t exist, or the query is broken
        continue


In [44]:
import hopsworks, src.config as c

# ── connect ─────────────────────────────────────────────
project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# ── point the view at the feature‑group we populated ────
fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = c.FEATURE_VIEW_NAME,       # "time_series_hourly_feature_view"
    version     = 1,                         # new v1 (store is empty now)
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print("✅ feature‑view created:", fv.name, "v", fv.version)


2025-05-10 10:10:46,073 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:10:46,080 INFO: Initializing external client
2025-05-10 10:10:46,082 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:10:47,359 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [45]:
NEW_VIEW_NAME = "citibike_hourly_feature_view"   # <‑ any unused name
fg = fs.get_feature_group("time_series_hourly_feature_group", version=1)

fv = fs.create_feature_view(
    name        = NEW_VIEW_NAME,
    version     = 1,
    description = "Hourly Citi Bike rides (3 top stations)",
    labels      = ["rides"],
    query       = fg.select_all()
)

print("✅ feature‑view created:", fv.name, "v", fv.version)


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [46]:
import hopsworks, src.config as c

project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# list everything the store still thinks exists
for fv in fs.get_feature_views(name=None):
    print(f"🟢  {fv.name:<35}  v{fv.version}")


2025-05-10 10:10:58,134 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:10:58,155 INFO: Initializing external client
2025-05-10 10:10:58,159 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:10:59,414 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215665/featurestores/1203294/featureview/None). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":120004,"usrMsg":"HTTP 404 Not Found","errorMsg":"Web application exception occurred"}', error code: 120004, error msg: Web application exception occurred, user msg: HTTP 404 Not Found

In [47]:
import hopsworks, src.config as c, json, requests

project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# the REST URL we need
base   = project._client._base_url          # e.g. https://c.app.hopsworks.ai
projId = project.id
storeId = fs.id

url = f"{base}/hopsworks-api/api/project/{projId}/featurestores/{storeId}/featureview"

resp = requests.get(url, headers=project._client._get_auth_header())
resp.raise_for_status()
views = resp.json()["items"]

print(f"Found {len(views)} Feature View objects in metadata:")
for v in views:
    print(" •", v["name"], "v", v["version"])


2025-05-10 10:11:01,850 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:11:01,871 INFO: Initializing external client
2025-05-10 10:11:01,874 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:11:03,115 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


AttributeError: 'Project' object has no attribute '_client'

In [48]:
 import os, requests, json, hopsworks, src.config as c

# ── 1 login just once — NO feature‑view code yet ───────────────────────────────
project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# ── 2 build the raw REST URL ───────────────────────────────────────────────────
base_url = project.client._base_url           # e.g. https://c.app.hopsworks.ai
proj_id  = project.id
store_id = fs.id

url = f"{base_url}/hopsworks-api/api/project/{proj_id}/featurestores/{store_id}/featureview"

# ── 3 GET every feature‑view object in JSON ────────────────────────────────────
hdrs = project.client._get_auth_header()
resp  = requests.get(url, headers=hdrs, timeout=60)
resp.raise_for_status()

views_json = resp.json()["items"]
print(f"\n📋  Registry currently contains {len(views_json)} Feature Views")
for v in views_json:
    print(" •", v["name"], "v", v["version"])


2025-05-10 10:11:06,192 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:11:06,205 INFO: Initializing external client
2025-05-10 10:11:06,207 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:11:07,436 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


AttributeError: 'Project' object has no attribute 'client'

In [49]:
import requests, hopsworks, src.config as c
from hopsworks import client as _hc

# ── 1 login once ──────────────────────────────────────────────────────────────
project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs         = project.get_feature_store()
raw_client = _hc.get_instance()          # global low‑level client

# ── 2 assemble REST URL + auth header (OLD helper) ────────────────────────────
base_url = raw_client._base_url          # e.g. https://c.app.hopsworks.ai
proj_id  = project.id
store_id = fs.id

url  = f"{base_url}/hopsworks-api/api/project/{proj_id}/featurestores/{store_id}/featureview"

hdrs = raw_client._get_auth_header()     # ← works on older SDKs

# ── 3 list every Feature‑View JSON still in the registry ──────────────────────
resp = requests.get(url, headers=hdrs, timeout=60)
resp.raise_for_status()

items = resp.json()["items"]
print(f"\n📋  Registry currently contains {len(items)} Feature Views:")
for v in items:
    print(f" • {v['name']:<35}  v{v['version']}")


2025-05-10 10:11:11,508 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:11:11,535 INFO: Initializing external client
2025-05-10 10:11:11,540 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 10:11:12,802 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


AttributeError: 'Client' object has no attribute '_get_auth_header'

In [ ]:
import hopsworks
import pandas as pd
import src.config as c  # ensure your config contains project name and API key

# 1. Connect to Hopsworks
project = hopsworks.login(
    project=c.HOPSWORKS_PROJECT_NAME,
    api_key_value=c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# 2. Load your final hourly dataframe (replace with your actual dataframe)
df = pd.read_parquet("data/hourly_data.parquet")  # adjust path as needed

# 3. Create feature group
fg = fs.create_feature_group(
    name=c.FEATURE_GROUP_NAME,  # e.g. "time_series_hourly_feature_group"
    version=1,
    description="Hourly aggregated Citi Bike rides per location",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

# 4. Save the data to the feature group
fg.insert(df, write_options={"wait_for_job": True})


2025-05-03 17:02:49,966 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 17:02:49,971 INFO: Initializing external client
2025-05-03 17:02:49,971 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 17:02:50,650 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


FileNotFoundError: [Errno 2] No such file or directory: 'data/hourly_data.parquet'